# Preparation: Streamlit UI and Streaming Structured Output

[code](https://github.com/alexeygrigorev/ai-bootcamp-codespace/tree/main/week4/code)

## Current Application Structure

This is our application so far:

`ver1.py`

In [ ]:
import asyncio

import search_agent

async def main():
    user_input = "How do I monitor data drift in production?"

    agent = search_agent.create_agent()
    callback = search_agent.NamedCallback(agent)

    result = await agent.run(user_input, event_stream_handler=callback)
    article = result.output

    print(article.format_article())

if __name__ == "__main__":
    asyncio.run(main())


Here the user is waiting for the agento to finish before seeing any result. We want to add streaming of the output to have a better user experience.

## The Streaming Challenge

We want the application to stream output instead of waiting for the agent to finish working.

The problem: we have structured output, so it's not as simple as streaming text. The output is JSON.

Let's implement streaming using PydanticAI's streaming capabilites. In PydanticAI, structured output hapens via a fictional tool call, so we need to detect when this tool is invoked.

`ver2.py`

In [ ]:
import asyncio

import search_agent

async def main():
    user_input = "How do I monitor data drift in production?"

    agent = search_agent.create_agent()
    callback = search_agent.NamedCallback(agent)

    previous_text = ""

    async with agent.run_stream(
        user_input, event_stream_handler=callback
    ) as result:
        async for item, last in result.stream_responses(debounce_by=0.01):
            for part in item.parts:
                if not hasattr(part, "tool_name"):
                    continue
                if part.tool_name != "final_result":
                    continue

                current_text = part.args
                delta = current_text[len(previous_text):]
                print(delta, end="", flush=True)
                previous_text = current_text

if __name__ == "__main__":
    asyncio.run(main())


This shows JSON, which is not very useful (yet), but we'll deal with it later.

If you want to get the final output this is how you do it (after the loop):

In [ ]:
article = await result.get_output()
print(article.format_article())

We can also get the messages:

In [ ]:
new_messages = result.new_messages()

## Incremental JSON Parsing

Now we can see the JSON as it comes in, but it's not very useful for the user. We need to parse the JSON as it arrives.

For that we can use a streaming JSON parser. For example, [JAXN](https://github.com/alexeygrigorev/jaxn). Alternatively you can use [streaming-json-parser](https://pypi.org/project/streaming-json-parser/) (which uses a different approach for parsing) or find some other implementation.

JAXN implements the same idea as SAX parsers for XML: you have a stream of incoming data, and as you process it, you get the data from the callbacks that react to the current state.

Let's install it:

In [1]:
!uv add jaxn

Resolved 263 packages in 1.25s                                       
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/4.63 KiB            
⠙ Preparing packages... (0/1)---------- 4.63 KiB/4.63 KiB           
Prepared 1 package in 176ms                                                  
Installed 1 package in 3ms                                  
 + jaxn==0.0.1


Import the necessary components:

In [ ]:
from jaxn import StreamingJSONParser, JSONParserHandler

You need the parser and the handler. The handler reacts to the current state in the parser, so we will use it for displaying the results.

We need to implement the methods for the handler. Here are the key callback methods:

on_field_start: Called when starting to read a field value

on_field_end: Called when a field value is complete

on_value_chunk: Called for each character as string values stream in

on_array_item_start: Called when starting a new object in an array

on_array_item_end: Called when finishing an object in an array

Our strategy:

Display title and section headers inside on_field_end

Display references in on_array_item_end

Display the content in a streaming way (print as it arrives)

Let's create the handler:



In [ ]:
class SearchResultArticleHandler(JSONParserHandler):
    def on_field_start(self, path: str, field_name: str):
        if field_name == "references":
            level = path.count("/") + 2
            print(f"\n{'#' * level} References\n")

    def on_field_end(self, path, field_name, value, parsed_value=None):
        if field_name == "title" and path == "":
            print(f"# {value}")

        elif field_name == "heading":
            print(f"\n\n## {value}\n")
        elif field_name == "content":
            print("\n") 

    def on_value_chunk(self, path, field_name, chunk):
        if field_name == "content":
            print(chunk, end="", flush=True)

    def on_array_item_end(self, path, field_name, item=None):
        if field_name == "references":
            title = item.get("title", "")
            filename = item.get("filename", "")
            print(f"- [{title}]({filename})")


This handler formats the streaming JSON into a readable article format as the data arrives.

## Creating the Parser

Now we create the parser with this handler:

In [ ]:
handler = SearchResultArticleHandler()
parser = StreamingJSONParser(handler)

And use it to parse incremental updates:

In [ ]:
parser.parse_incremental(delta)

When we run it, we see the output appearing on the screen in a more structured way, with proper formatting and real-time updates.

This is our `ver3.py` file.

In [ ]:
import asyncio

from time import time
from typing import Any, Dict
from jaxn import JSONParserHandler, StreamingJSONParser

import search_agent

from agent_logging import log_streamed_run, save_log

class SearchResultArticleHandler(JSONParserHandler):
    
    def on_field_start(self, path: str, field_name: str) -> None:
        if field_name == "references":
            header_level = path.count('/') + 2
            print(f"\n\n{'#' * header_level} References\n")
    
    def on_field_end(self, path: str, field_name: str, value: str, parsed_value: Any = None) -> None:
        if field_name == "title" and path == "":
            print(f"# {value}\n")
        
        if field_name == "heading":
            print(f"\n\n## {value}\n")
    
    def on_value_chunk(self, path: str, field_name: str, chunk: str) -> None:
        if field_name == "content":
            print(chunk, end="", flush=True)
    
    def on_array_item_end(self, path: str, field_name: str, item: Dict[str, Any] = None) -> None:
        if field_name == "references":
            print(f"- [{item['title']}]({item['filename']})")


async def main():
    user_input = "How do I monitor data drift in production?"

    agent = search_agent.create_agent()
    callback = search_agent.NamedCallback(agent)

    # result = await agent.run(user_input, event_stream_handler=callback)
    # article = result.output

    handler = SearchResultArticleHandler()
    parser = StreamingJSONParser(handler)

    previous_text = ""

    async with agent.run_stream(
        user_input, event_stream_handler=callback
    ) as result:
        async for item, last in result.stream_responses(debounce_by=0.01):
            for part in item.parts:
                if not hasattr(part, "tool_name"):
                    continue
                if part.tool_name != "final_result":
                    continue

                current_text = part.args
                delta = current_text[len(previous_text):]
                parser.parse_incremental(delta)
                previous_text = current_text

        log_entry = await log_streamed_run(agent, result)
        save_log(log_entry)


    # print(article.format_article())


if __name__ == "__main__":
    asyncio.run(main())

## Building the Streamlit Application

Let's now turn this into a Streamlit chat application.

My prompt for ChatGPT:

```text
Create a streamlit chat application that asks the user for input and sends it to the agent.

Display the tool calls from the agent as well as the output as it arrives.

Here's the current code I have:
# insert the tool call callback code
# insert the streaming code
```

[Converstaion with ChatGPT](https://chatgpt.com/share/69049307-ef70-800a-9195-428f0c53e7f0)

The resulting Streamlit application provides:

- A chat interface for user input
- Real-time display of tool calls
- Streaming output as the agent generates responses
- Proper formatting of the structured output

In [ ]:
!uv add streamlit
!uv run streamlit run ver4.py

## Next Steps

Now our application is ready for the next step: collecting logs. The streaming interface provides a much better user experience while maintaining all the functionality of our agent system.